In [1]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np
import scipy
from scipy.stats import beta, binom, entropy
import random
import json
import copy
import math
import pickle
import statistics

# my modules
import scientist
import evaluation
import helper
import settings
import publisher

# global variables
num_bins = 30
num_draws = 10
num_participants = 10
num_generations = 15

In [2]:
# distribution of bins
bins_to_probs = {}
for i in range(0, num_bins):
    bins_to_probs[i] = 0.5

In [3]:
def make_participants(setting, alpha_value):
    participants = []

    for i in range (0, num_participants):
        if setting == "rate":
            report_set = settings.ReportingSetting("rate")
        elif setting == "data":
            report_set = settings.ReportingSetting("data")
        elif setting == "subset":
            report_set = settings.ReportingSetting("subset")

        # make participant
        participant = scientist.Participant(alpha=alpha_value, reporting_setting=report_set)
                        
        participants.append(participant)

    return(participants)

In [4]:
def run_experiment(setting, alpha_value, rel_pl_data_val, rel_pl_surprise_val, rel_pl_bias_val):
    # each experiment starts with a blank cannon (starts with 1-1 prior)
    scientific_record = {}
    for bin_num in range(0, num_bins):
        scientific_record[bin_num] = {} 
        scientific_record[bin_num][0] = 1
        scientific_record[bin_num][1] = 1
    
    for generation in range(0, num_generations):
#         print(f"\n* Generation {generation}...")
#         helper.print_record(scientific_record, num_bins)
#         print(f"   Arm score: {evaluation.arm_parameter_score(scientific_record, bins_to_probs)}")
#         print(f"   Entropy score: {evaluation.total_entropy_score(scientific_record, bins_to_probs)}")
        
        # each generation gets an entirely new set of participants
        participants = make_participants(setting, alpha_value)

        # scientists explore and submit reports
        for participant in participants:
            # sample
            for i in range(0, num_draws):
                bin_number, value = participant.sample(scientific_record, num_bins, bins_to_probs)
                
#                 print(f"   sample from bin {bin_number}: {value}")

            # choose the bin
            bin_choice = participant.choose_bin(scientific_record, num_bins, num_draws)
#             print(f"   chose bin {bin_choice}")

            # make a report
            participant.report(num_bins, num_draws)
            
        # the peer review board selects reports for publication and returns the updated scientific record
        scientific_record = publisher.peer_review(participants, scientific_record, rel_pl_data_val, rel_pl_surprise_val, rel_pl_bias_val, num_draws)
        
#     print("\n\n* FINAL RESULTS")
#     helper.print_record(scientific_record, num_bins)
    
#     # final metric of how well scientists play the multi-armed bandit game
#     print(evaluation.arm_parameter_score(scientific_record, bins_to_probs))

#     # final metric of how well scientists reduce the entropy of the scientific record
#     print(evaluation.total_entropy_score(scientific_record, bins_to_probs))
    
    return(evaluation.arm_parameter_score(scientific_record, bins_to_probs), evaluation.total_entropy_score(scientific_record, bins_to_probs))

In [5]:
# setting, alpha_value, rel_pl_data_val, rel_pl_surprise_val, rel_pl_bias_val
run_experiment("data", 0, 1, 1, 0)

(0.2104208754208754, 0.13533164344937282)

In [6]:
# scale over amount of supporting data 
rel_pl_data = np.linspace(0, 10, 21)

# scale over how surprising the data is
rel_pl_surprise = np.linspace(0, 10, 21)

# rate of bump for publication bias (0.01 = 1% publication bias)
rel_pl_bias = np.linspace(0, 1, 21)

In [7]:
publishing_policies_space = {}
exp_no = 0

for rel_pl_data_val in rel_pl_data:
    print(f"examining value: {rel_pl_data_val}")
    for rel_pl_surprise_val in rel_pl_surprise:
        print(f"   surprise value: {rel_pl_surprise_val}")
        for rel_pl_bias_val in rel_pl_bias:
            total_arm_score = 0
            total_entropy_score = 0
            vals = []

            for i in range(0, 5):
                arm_score, entropy_score = run_experiment("data", 0, rel_pl_data_val, rel_pl_surprise_val, rel_pl_bias_val)
                total_arm_score += arm_score
                total_entropy_score += entropy_score
                vals.append(total_entropy_score)

            std_dev = statistics.stdev(vals)
                
            key = (rel_pl_data_val, rel_pl_surprise_val, rel_pl_bias_val)
            publishing_policies_space[key] = [total_arm_score / 5, total_entropy_score / 5, std_dev] # average over 10 runs of each combination
            exp_no += 1

# save the results
pickle.dump(
    publishing_policies_space,
    open(f"/Users/marinamancoridis/Thesis/Thesis_Simulations/publishing_policies_30_bins_d2_with_5.p", "wb")
)

examining value: 0.0
   surprise value: 0.0
   surprise value: 0.5
   surprise value: 1.0
   surprise value: 1.5
   surprise value: 2.0
   surprise value: 2.5
   surprise value: 3.0
   surprise value: 3.5
   surprise value: 4.0
   surprise value: 4.5
   surprise value: 5.0
   surprise value: 5.5
   surprise value: 6.0
   surprise value: 6.5
   surprise value: 7.0
   surprise value: 7.5
   surprise value: 8.0
   surprise value: 8.5
   surprise value: 9.0
   surprise value: 9.5
   surprise value: 10.0
examining value: 0.5
   surprise value: 0.0
   surprise value: 0.5
   surprise value: 1.0
   surprise value: 1.5
   surprise value: 2.0
   surprise value: 2.5
   surprise value: 3.0
   surprise value: 3.5
   surprise value: 4.0
   surprise value: 4.5
   surprise value: 5.0
   surprise value: 5.5
   surprise value: 6.0
   surprise value: 6.5
   surprise value: 7.0
   surprise value: 7.5
   surprise value: 8.0
   surprise value: 8.5
   surprise value: 9.0
   surprise value: 9.5
   surprise va

   surprise value: 2.0
   surprise value: 2.5
   surprise value: 3.0
   surprise value: 3.5
   surprise value: 4.0
   surprise value: 4.5
   surprise value: 5.0
   surprise value: 5.5
   surprise value: 6.0
   surprise value: 6.5
   surprise value: 7.0
   surprise value: 7.5
   surprise value: 8.0
   surprise value: 8.5
   surprise value: 9.0
   surprise value: 9.5
   surprise value: 10.0
examining value: 8.5
   surprise value: 0.0
   surprise value: 0.5
   surprise value: 1.0
   surprise value: 1.5
   surprise value: 2.0
   surprise value: 2.5
   surprise value: 3.0
   surprise value: 3.5
   surprise value: 4.0
   surprise value: 4.5
   surprise value: 5.0
   surprise value: 5.5
   surprise value: 6.0
   surprise value: 6.5
   surprise value: 7.0
   surprise value: 7.5
   surprise value: 8.0
   surprise value: 8.5
   surprise value: 9.0
   surprise value: 9.5
   surprise value: 10.0
examining value: 9.0
   surprise value: 0.0
   surprise value: 0.5
   surprise value: 1.0
   surprise v

KeyboardInterrupt: 